In [1]:
import segmentation_models_pytorch as smp
from trainer import Trainer

# Training

In [2]:
# def TRAIN(MODEL, ENCODER, OPTIMIZER, LOSS):
#     if(MODEL == 'Unet'):
#         model = smp.Unet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
#     elif(MODEL == 'FPN'):
#         model = smp.FPN(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
#     elif(MODEL == 'Linknet'):
#         model = smp.Linknet(ENCODER, encoder_weights='imagenet', classes=5, activation=None)
    
#     model_trainer = Trainer(model = model, optim = OPTIMIZER, loss = LOSS, lr = 1e-3, bs = 8, name = ENCODER+'_'+MODEL+'_'+LOSS+'_'+OPTIMIZER)
#     model_trainer.seed_everything(43)
#     model_trainer.do_cutmix = False
#     model_trainer.freeze()
#     model_trainer.change_loader(crop_type=0, shape=512)
#     model_trainer.fit(10)
#     model_trainer.do_cutmix = True
#     model_trainer.unfreeze()
#     model_trainer.change_loader(crop_type=0, shape=512)
#     model_trainer.fit(20)
#     model_trainer.do_cutmix = True
#     model_trainer.unfreeze()
#     model_trainer.change_loader(crop_type=1, shape=512)
#     model_trainer.fit(20)
#     model_trainer.do_cutmix = False
#     model_trainer.freeze()
#     model_trainer.change_loader(crop_type=1, shape=512)
#     model_trainer.fit(5)

In [3]:
# # Accordingly append to the list and train multiple models
# # Naming has also been adjusted 
# # ['Ranger','Over9000'] * ['Dice', 'BCE+DICE', 'BCE+DICE+JACCARD']
# Model = ['FPN']
# Encoder = ['efficientnet-b4']
# Optimizer = ['Ranger', 'Over9000']
# Loss = ['DICE', 'BCE+DICE', 'BCE', 'BCE+DICE+JACCARD']
# for model in Model:
#     for encoder in Encoder:
#         for optimizer in Optimizer:
#             for loss in Loss:
#                 TRAIN(model, encoder, optimizer, loss)

# Validate

In [4]:
model = smp.FPN('efficientnet-b4', encoder_weights='imagenet', classes=5, activation=None)
model_trainer =Trainer(model = model, optim = 'Ranger', loss = 'BCE+DICE', lr = 1e-3, bs = 8, name = 'none')
# model_trainer.load_model('efficientnet-b4_FPN_BCE+DICE_Over9000_best_dice.pth')

In [5]:
# model_trainer.validate()

In [ ]:
Model = ['FPN']
Encoder = ['efficientnet-b4']
Optimizer = ['Ranger', 'Over9000']
Loss = ['DICE', 'BCE+DICE', 'BCE', 'BCE+DICE+JACCARD']
for model in Model:
    for encoder in Encoder:
        for optimizer in Optimizer:
            for loss in Loss:
                print(model, encoder, optimizer, loss)
                model_trainer.load_model(encoder+'_'+model+'_'+loss+'_'+optimizer+'_best_lb_metric.pth')
                model_trainer.validate()

FPN efficientnet-b4 Ranger DICE
Loaded model with dice:  0.37721314024232044
Starting epoch: 1 | phase: val | ⏰: 12:04:16


# Testing

In [ ]:
model = smp.FPN('efficientnet-b4', encoder_weights='imagenet', classes=5, activation=None)
model_trainer =Trainer(model = model, optim = 'Ranger', loss = 'BCE+DICE', lr = 1e-3, bs = 8, name = 'none')
model_trainer.load_model('efficientnet-b4_FPN_BCE+DICE_Over9000_best_dice.pth')

In [ ]:
model_trainer.predict()

In [ ]:
import tifffile as tif

In [ ]:
sam = tif.imread('../EndoCV2020_testSubmission/semantic_masks/EAD2020_S000_0.tif')

In [ ]:
sam.shape

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(sam[4])

In [ ]:
img = plt.imread('../ead2020/SemanticSegmentation/EAD2020_S000_0.jpg')

In [ ]:
plt.imshow(img)

In [ ]:
img.shape

In [ ]:
sam.max()